# Job Description of Analysis

**Author**: Jerome Wynne (jeromewynne@das-ltd.co.uk)

**Created**: 5th November 2020  

**Environment**: wmchack

**Summary**: Classifying job roles: An NLP challenge using NHS Job descriptions, matching to Professional groups to (i.e. using FEDIP & AphA definition of analytical or civil service GORS/GSS/GDS).

In [ ]:
import pandas as pd


In [ ]:
def load_job_descriptions():
    ''' Reads all .json files in /data into dataframe.

        The dataframe constructed can be viewed in /logs/info.log.

        Returns:
            pd.DataFrame: dataframe where each row corresponds to one 
                            file.
    '''
    fps = glob(r'..\data\*.json')
    fps = [fp for fp in fps if 'example' not in fp] # remove example.json
    list_of_page_dct = []
    for fp in fps: # read all of the files into a list of dicts
        with open(fp, 'r', encoding='utf-8') as f:
            list_of_page_dct += [json.load(f)]
    
    # flatten the list of dicts to a DataFrame
    df = pd.json_normalize(list_of_page_dct)

    # write df to log
    logging.info('\n' + df.to_string())
    return(df)